# ch09_AST_Hy_Lisp

## AST 덤프

In [1]:
import ast
ast.dump(ast.parse("x = 42"))

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=42))])"

## 컴파일하고 실행하기

In [2]:
babo

NameError: name 'babo' is not defined

In [3]:
code_object = compile(ast.parse("babo=42"), '<input>', 'exec')
eval(code_object)

In [4]:
babo

42

## AST로 프로그래밍하기 

In [10]:
# 변수
hello_world = ast.Str(s='hello world', lineno=1, col_offset=1)

# 함수 정의
print_name = ast.Name(id='print', ctx=ast.Load(), lineno=1, col_offset=1)
print_call = ast.Call(func=print_name, ctx=ast.Load(), args=[hello_world], keywords=[], lineno=1, col_offset=1)

module = ast.Module(body=[ast.Expr(print_call, lineno=1, col_offset=1)], lineno=1, col_offset=1, type_ignores=[])

code = compile(module, '', 'exec')
eval(code)

hello world


## AST 탐색하기 -  ex) 연산자 변경

In [13]:
import ast

tree = ast.parse('x = 1/3')
ast.fix_missing_locations(tree)
eval(compile(tree, '', 'exec'))
print(ast.dump(tree))
print(x)

Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=BinOp(left=Num(n=1), op=Div(), right=Num(n=3)))])
0.3333333333333333


In [15]:
class ReplaceBinOp(ast.NodeTransformer):
    """ 모든 이진 연산자를 더하기 연산자로 교체"""
    def visit_BinOp(self, node):
        return ast.BinOp(left=node.left, op=ast.Add(), right=node.right)
    
tree = ReplaceBinOp().visit(tree)
ast.fix_missing_locations(tree)
eval(compile(tree, '', 'exec'))
print(ast.dump(tree))
print(x)

Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=BinOp(left=Num(n=1), op=Add(), right=Num(n=3)))])
4


## 코드 검사에 활용

ex) class에 정적 선언으로 했었어야 하는 코드 발견 - 일반메소드인데 self를 사용하지 않은 경우

In [93]:
class Bad(object):
    def foo(self, a):
        return a
    

class Good_general(object):
    def __init(v):
        self.v = v
    def foo(self, a):
        return self.v + a  
    
class Good_static(object):
    @staticmethod
    def foo(a):
        return a

In [99]:

def contain_self_arg(body_item):
    # 정적이지 않다면 self argument 포함여부 조사
    try:
        first_arg = body_item.args.args[0]
    except IndexError:
        return False
    
    for func_stmt in ast.walk(body_item):
        if(isinstance(func_stmt, ast.Name) and first_arg.arg == func_stmt.id):
            # self 인수 사용 확인
            print(f'\t\tparam= {first_arg.arg}, {func_stmt.id}')
            return True
    else:
        return False
    
class StaticmethodChecker(object):
    def __init__(self, tree):
        self.tree = tree
        
    def run(self):
        for stmt in ast.walk(self.tree):
            #  클래스가 아닌 것은 무시
            if not isinstance(stmt, ast.ClassDef):
                #print('skip: ', stmt)
                continue
            
            print(f'examine {stmt.name}')
            
            # 클래스이면서 메소드를 찾기 위해 클래스 맴버를 반복 탐색 
            for body_item in stmt.body:
                # 메소드가 아니면 넘어감
                if not isinstance(body_item, ast.FunctionDef):
                    #print('skip: ', body_item)
                    continue
            
                # 데코레이터를 사용했는지 확인
                if(len(body_item.decorator_list) > 0):                
                    for decorator in body_item.decorator_list:
                        if (isinstance(decorator, ast.Name) and decorator.id == 'staticmethod'):
                            # 정적이라면 검사 대상이 아님
                            print(f'\t{body_item.name}, {decorator.id} found.. OK')
                            break
                        else:
                            # 정적이지 않다면 self argument 포함여부 조사
                            if not contain_self_arg(body_item):
                                print(f"\t{body_item.name}, [ERROR] not found self")                            
                else:
                    # 데코레이터가 없는 경우 
                    if not contain_self_arg(body_item):
                        print(f"\t{body_item.name}, [ERROR] not found self")                                                            
                                

In [100]:
code = """
class Bad(object):
    def foo(self, a):
        return a
    
class Good_general(object):
    def __init(v):
        self.v = v
    def foo(self, a):
        return self.v + a  
    
class Good_static(object):
    @staticmethod
    def foo(a):
        return a

"""


StaticmethodChecker(ast.parse(code)).run()

examine Bad
	foo, [ERROR] not found self
examine Good_general
		param= v, v
		param= self, self
examine Good_static
	foo, staticmethod found.. OK


## Hy 살펴보기

In [ ]:
=> (+ 1 2)
3

In [ ]:
=> (defn hello [name]
... (print (% "hello world to %s" name)))
=> (hello "hoondori")
hello world to hoondori

In [ ]:
=> (import uuid)
=> (uuid.uuid4)
UUID('9e211323-9dac-4871-8cd7-685dc5f8c80a')